## Get ETH Active Agents by Date
ETH Active Agents are addresses with a balance of more than 0.01 ETH

In [1]:
import pandas as pd
from google.cloud import bigquery, bigquery_storage
from google.oauth2 import service_account

final_ts = '2021-09-01 00:00:00'
minimum_balance = 0.01


def get_df(query, key_path: str = "bigquery_project.json") -> pd.DataFrame:
    """
    Get Pandas DataFrame by SQL query
    :param key_path: path of bigquery key file
    :param query: SQL query to get data for a DataFrame
    :return: DataFrame
    """
    # Construct a BigQuery client object.
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],)
    bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
    bq_storage_client = bigquery_storage.BigQueryReadClient(credentials=credentials)
    # Execute query
    return bq_client.query(query).result().to_dataframe(bqstorage_client=bq_storage_client)

In [ ]:
query_1 = f'''
WITH double_entry_book AS (
     -- debits
     SELECT
          to_address AS address,
          DATE(block_timestamp) as block_date,
          value as value
     FROM `bigquery-public-data.crypto_ethereum.traces`
     WHERE block_timestamp < '{final_ts}'
       AND to_address is not null
       AND status = 1
       AND (call_type not in ('delegatecall', 'callcode', 'staticcall') OR call_type is null)
     UNION ALL
     -- credits
     SELECT
          from_address AS address,
          DATE(block_timestamp) as block_date,
          -value AS value
     FROM `bigquery-public-data.crypto_ethereum.traces`
     WHERE block_timestamp < '{final_ts}'
       AND from_address is not null
       AND status = 1
       AND (call_type not in ('delegatecall', 'callcode', 'staticcall') OR call_type is null)
     UNION ALL
     -- transaction fees debits
     SELECT
          miner AS address,
          DATE(block_timestamp) as block_date,
          cast(receipt_gas_used AS numeric) * cast(gas_price AS numeric) AS value
     FROM `bigquery-public-data.crypto_ethereum.transactions` AS transactions
     INNER JOIN `bigquery-public-data.crypto_ethereum.blocks` AS blocks ON blocks.number = transactions.block_number
     WHERE block_timestamp < '{final_ts}'
     UNION ALL
     -- transaction fees credits
     SELECT
          from_address AS address,
          DATE(block_timestamp) as block_date,
          -(cast(receipt_gas_used AS numeric) * cast(gas_price AS numeric)) AS value
     FROM `bigquery-public-data.crypto_ethereum.transactions`
     WHERE block_timestamp < '{final_ts}'
),
-- zero balances for correct calculation
zero_balances as (
    SELECT
        address,
        block_date,
        0 as value
    FROM (
            SELECT
                address,
                min(block_date) as min_block_date,
                max(block_date) as max_block_date,
                sum(value) as final_value
            FROM double_entry_book
            GROUP BY address) as addresses,
          UNNEST(GENERATE_DATE_ARRAY('2015-07-30', DATE('{final_ts}'))) AS block_date
    WHERE block_date>=min_block_date
      AND (final_value > 0.01 OR block_date <= max_block_date)
),
double_entry_book_with_zero_balances as (
     SELECT
          address,
          block_date,
          value
     FROM double_entry_book

     UNION ALL

     SELECT
          address,
          block_date,
          value
     FROM zero_balances
)
SELECT
     block_date,
     sum(eth_balance) as total_eth_balance,
     count(address) as number_of_addresses
FROM (
     SELECT
          address,
          block_date,
          sum(eth_balance_change_daily) OVER (PARTITION BY address ORDER BY block_date) AS eth_balance
     FROM (
          SELECT
               address,
               block_date,
               sum(value)/1e18  AS eth_balance_change_daily
          FROM double_entry_book_with_zero_balances
          GROUP BY address, block_date)
     )
WHERE eth_balance > {minimum_balance}
GROUP BY block_date
ORDER BY block_date
'''
active_agents_by_date_df = get_df(query_1)
active_agents_by_date_df.to_csv('data/eth_active_agents.csv')
print(active_agents_by_date_df.head())